In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
from sklearn.preprocessing import OneHotEncoder

In [3]:
tf.reset_default_graph()

In [4]:
# Load training and test data
((train_data, train_labels),(test_data, test_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
test_data = test_data/np.float32(255)

ohe=OneHotEncoder(sparse=False, categories='auto')
train_labels=ohe.fit_transform(train_labels.reshape(-1,1))
test_labels=ohe.fit_transform(test_labels.reshape(-1,1))

In [5]:
x = tf.placeholder(tf.float32, shape=[None,28,28], name='X')
x_img = tf.reshape(x, [-1,28,28,1])
y_label = tf.placeholder(tf.float32, shape=[None, 10], name='y_label')
y_true = tf.argmax(y_label, axis=1)

rate = tf.placeholder(tf.float32, name='rate')

# Layer Functions

In [6]:
def conv_layer(inp, num_inp_channels, filter_size, num_filters):
    shape = [filter_size, filter_size, num_inp_channels, num_filters]
    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    biases = tf.Variable(tf.constant(0.1, shape=[num_filters]))
    layer = tf.nn.conv2d(input=inp, filter=weights, strides=[1,1,1,1], padding='SAME')
    layer+= biases
    return layer

In [7]:
def pool_layer(inp):
    layer = tf.nn.max_pool(value=inp, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    return layer

In [8]:
def fc_layer(inp, num_inps, num_outs):
    weights = tf.Variable(tf.truncated_normal([num_inps, num_outs], stddev=0.1))
    biases = tf.Variable(tf.constant(0.1, shape=[num_outs]))
    return tf.matmul(inp,weights) + biases

# Create CNN

In [9]:
conv1 = tf.nn.relu(conv_layer(inp=x_img, num_inp_channels=1, filter_size=5, num_filters=6))
pool1 = pool_layer(conv1)
conv2 = tf.nn.relu(conv_layer(inp=pool1, num_inp_channels=6, filter_size=5, num_filters=16))
pool2 = pool_layer(conv2)

num_features = pool2.shape[1:].num_elements()
flat_layer = tf.reshape(pool2, [-1, num_features])

fc1 = tf.nn.relu(fc_layer(flat_layer, num_inps=num_features, num_outs=128))
fc2 = fc_layer(fc1, num_inps=128, num_outs=10)

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
y_pred = tf.nn.softmax(fc2)
y_pred_val = tf.argmax(y_pred, axis=1)

# Cost Function

In [17]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc2, labels=y_label)
cost = tf.reduce_mean(cross_entropy)

In [18]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

# Accuracy

In [19]:
correct_pred = tf.equal(y_pred_val, y_true)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [20]:
num_epochs = 10
batch_size = 100

# TF session

In [21]:
from time import time

In [22]:
saver = tf.train.Saver()

In [27]:
def batch_gen(batch_size):
    cut=np.random.randint(0,train_labels.shape[0]-batch_size)
    return train_data[cut:cut+batch_size], train_labels[cut:cut+batch_size]

In [29]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    try:
        saver.restore(sess,"Model/model.ckpt")
        print("[*] Model Restored.")
    except:
        print("[!] Probably encountered error in restoring.")
    totl = train_labels.shape[0]
    try:
        for epoch in range(num_epochs):
            strt=time()
            train_acc = 0
            print("Epoch:",(epoch+1),"/",num_epochs)
            for batch in range(0,totl,batch_size):
                print("\rProgress: %.2f" % (batch*100/totl),'%', end='')
                x_batch, y_label_batch = batch_gen(batch_size)
                feed_dict = {x:x_batch, y_label: y_label_batch, rate: 0.2}
                sess.run(optimizer, feed_dict=feed_dict)
                train_acc += sess.run(accuracy, feed_dict=feed_dict)

            print("\rProgress: 100.00 %")
            train_acc /= int(len(train_labels)/batch_size)
            print("Time:",(time()-strt))
            print("Training Accuracy:",train_acc*100)
            feed_dict = {x:test_data, y_label: test_labels, rate: 0}
            test_acc = sess.run(accuracy, feed_dict=feed_dict)
            print("Testing Accuracy:",test_acc*100,'\n')
    except KeyboardInterrupt:
        pass
    saver.save(sess,"Model/model.ckpt")
    print("\n[*] Model Saved.")

INFO:tensorflow:Restoring parameters from Model/model.ckpt
[*] Model Restored.
Epoch: 1 / 10
Progress: 100.00 %
Time: 41.82014060020447
Training Accuracy: 99.76666687925658
Testing Accuracy: 99.04000163078308 

Epoch: 2 / 10
Progress: 35.17 %
[*] Model Saved.
